<a href="https://colab.research.google.com/github/bestarch/Redis-Workshops/blob/main/11-LLM_GCP_Workshop/GCP_GenAI_demo_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd
!pip install --upgrade pip

# Install required libraries
!pip install -U langchain gradio
!pip install -U langchain-core
!pip install -U langchain-google-vertexai
!pip install jupyterlab_widgets
!pip install -U langcache
%pip install -qU pypdf


/Users/abhishek.srivastava/apps/workspace/redis-ai/chatpdf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 19.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.4
    Uninstalling gradio_client-1.10.4:
      Successfully uninstalled gradio_client-1.10.4
  Attempting uninstall: gradio
    Found existing installation: gradio 5.37.0
    Uninstalling gradio-5.37.0:
      Successfully uninstalled gradio-5.37.090m━━━━━━━━━━━━━━━━━━━ 1/2 [gradio]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [gradio]2m1/2 [gradio]
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.68
    Uninstalling langchain-core-0.3.68:
      Successfully uninstalled langchain-core-0.3.68
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!wget https://storage.googleapis.com/abhi-data-2024/how_india_shops_online.pdf -O report.pdf


In [1]:
#!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2
import os
import subprocess

def check_and_authenticate():
    adc_path = os.path.expanduser("~/.config/gcloud/application_default_credentials.json")
    if os.path.exists(adc_path):
        print("✅ Already authenticated.")
    else:
        print("🔑 Not authenticated. Starting login flow...")
        subprocess.run(["gcloud", "auth", "application-default", "login"])

check_and_authenticate()

✅ Already authenticated.


In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings
from getpass import getpass
import vertexai
#import google.auth
import vertexai

# input your GCP project ID and region for Vertex AI
PROJECT_ID = 'central-beach-194106'#getpass("PROJECT_ID:")
REGION = 'us-central1' #input("REGION:")

#creds, _ = google.auth.default(quota_project_id=PROJECT_ID)


#vertexai.init(project=PROJECT_ID, credentials=creds, location=REGION)
vertexai.init(project=PROJECT_ID, location=REGION)

print(f'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}')



In [8]:
### Following code only works when running the notebook in Google Colab or GCP vertex AI Notebook
'''
from langchain_google_vertexai import VertexAIEmbeddings
from google.colab import auth
from getpass import getpass
import vertexai
import google.auth
import vertexai

# input your GCP project ID and region for Vertex AI
PROJECT_ID = getpass("PROJECT_ID:")
REGION = 'us-central1' #input("REGION:")

auth.authenticate_user()

#creds, _ = google.auth.default(quota_project_id=PROJECT_ID)
print('Authenticated')

#vertexai.init(project=PROJECT_ID, credentials=creds, location=REGION)
vertexai.init(project=PROJECT_ID, location=REGION)

print(f'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}')

'''

'\nfrom langchain_google_vertexai import VertexAIEmbeddings\nfrom google.colab import auth\nfrom getpass import getpass\nimport vertexai\nimport google.auth\nimport vertexai\n\n# input your GCP project ID and region for Vertex AI\nPROJECT_ID = getpass("PROJECT_ID:")\nREGION = \'us-central1\' #input("REGION:")\n\nauth.authenticate_user()\n\n#creds, _ = google.auth.default(quota_project_id=PROJECT_ID)\nprint(\'Authenticated\')\n\n#vertexai.init(project=PROJECT_ID, credentials=creds, location=REGION)\nvertexai.init(project=PROJECT_ID, location=REGION)\n\nprint(f\'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}\')\n\n'

In [2]:
from langchain import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List


# Define your desired data structure.
class PromptResponse(BaseModel):
    prompt: str = Field(description="User input question about information in the document.")
    response: str = Field(description="Grounded answer from the LLM pertaining to the user's question.")

class FAQs(BaseModel):
    pairs: List[PromptResponse] = Field(description="List of prompt response pairs extracted from the document")


# Set up a parser + inject instructions into the prompt template.
json_parser = JsonOutputParser(pydantic_object=FAQs)


In [3]:
from langchain_google_vertexai import VertexAI

# Create LLM
llm = VertexAI(
    model_name="gemini-2.5-pro",
    max_output_tokens=2048,
    temperature=0.5,
    verbose=True,
)

prompt = PromptTemplate(
    template="""You are a document intelligence tool used to extract FAQs
    from portions of a source PDF document. Put yourself in the shoes of a potential
    reader of the provided material and anticipate what questions they might have.
    Looking at the context below, your goal is to pull out some relevant question (prompt) 
    and answer (response) pairs, using only what's provided.

    Other Rules:
    - Create as many FAQs as possible, even rewording the same question and answer
    a few times while remaining faithful to the truth and provided content.
    - Ignore heavy marketing or salesly concepts and focus specifically on
    factual data.
    - Do not make up any information, only use what is provided in the context.
    
    {format_instructions}

    Document Context:\n{doc}\n""",
    input_variables=["doc"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)

faq_generator_chain = prompt | llm | json_parser

In [ ]:

from langcache import LangCache

LANG_CACHE_SERVER_URL = "https://gcp-us-east4.langcache.redis.io"
LANG_CACHE_ID = "****"
LANG_CACHE_SERVICE_KEY = "******"

with LangCache(
    server_url=LANG_CACHE_SERVER_URL,
    cache_id=LANG_CACHE_ID,
    service_key=LANG_CACHE_SERVICE_KEY,
) as lang_cache:

    res = lang_cache.set(prompt="How does semantic caching work?", response="Semantic caching stores and retrieves data based on meaning, not exact matches.")

    # Handle response
    print(res)


ServiceUnavailableErrorResponseContent: {"detail":"Request was rate limited by the embedding provider, or the quota was exceeded","status":503,"title":"Rate Limited By Embedding Provider","type":"/errors/embeddings/too-many-requests"}

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

file = "report.pdf"

# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=50, add_start_index=True
)

loader = PyPDFLoader(file)
documents = loader.load()

chunks = text_splitter.split_documents(documents)
chunked_docs = [doc.page_content for doc in chunks]

In [41]:
print(f"Total number of chunks: {len(chunked_docs)}")
print(chunked_docs[11])

Total number of chunks: 96
explore digital avenues.  
4 https://www.businesstoday.in/latest/corporate/story/indians-spent-87-billion-hours-on-shopping-apps-in-2022 report-366927-2023-01-20


In [ ]:

faqs=[]  
res = faq_generator_chain.invoke({"doc": chunked_docs[12]})
if res:
    faqs.extend(res['pairs'])
    print(f"Total number of FAQs generated: {len(res['pairs'])}")
    print(f"Sample FAQ: {res['pairs'][0]}")
else:
    print("No FAQs generated or unexpected response format.")

print(faqs)

Total number of FAQs generated: 6
Sample FAQ: {'prompt': 'What are the common reasons why people in India shop online?', 'response': 'Both urban dwellers and consumers from the rest of India make planned purchases online to access a wider range of price options and brands. Additionally, for both groups, browsing online shopping platforms during their free time can trigger a purchase.'}
[{'prompt': 'What are the common reasons why people in India shop online?', 'response': 'Both urban dwellers and consumers from the rest of India make planned purchases online to access a wider range of price options and brands. Additionally, for both groups, browsing online shopping platforms during their free time can trigger a purchase.'}, {'prompt': "What are the main motivators for consumers in the 'rest of India' to shop online?", 'response': "The primary reasons consumers in the 'rest of India' opt for online shopping are the absence of physical stores for premium brands, stockouts of certain prod

In [ ]:
from langchain_core.outputs import Generation
from langchain_redis import RedisVectorStore
from langchain_redis.cache import EmbeddingsVectorizer
from langcache import LangCache


#vectorizer = EmbeddingsVectorizer(embeddings=model)
questions = [faq["prompt"] for faq in faqs]
for i, faq in enumerate(faqs):
	print(f"Processing faq {i+1} of {len(faqs)}", flush=True)
	#cache.update(faq["prompt"], "llm1", [Generation(text=faq["response"])])
	with LangCache(
		server_url=LANG_CACHE_SERVER_URL,
		cache_id=LANG_CACHE_ID,
		service_key=LANG_CACHE_SERVICE_KEY,
	) as lang_cache:

		res = lang_cache.set(prompt=faq["prompt"], response=faq["response"])
	

Processing faq 1 of 6
Processing faq 2 of 6
Processing faq 3 of 6
Processing faq 4 of 6
Processing faq 5 of 6
Processing faq 6 of 6


In [ ]:
%%time
from langcache import LangCache


result = None
for question in questions:
   print(f"Question: {question}")
   with LangCache(
		server_url=LANG_CACHE_SERVER_URL,
		cache_id=LANG_CACHE_ID,
		service_key=LANG_CACHE_SERVICE_KEY,
	) as lang_cache:

      result = lang_cache.search(prompt=question, similarity_threshold=0.9)
   if result:
      print(f"Cached response: {result}\n")
   else:
      print(f"Cache miss for question: {question}")


Question: What are the common reasons why people in India shop online?
Cached response: Both urban dwellers and consumers from the rest of India make planned purchases online to access a wider range of price options and brands. Additionally, for both groups, browsing online shopping platforms during their free time can trigger a purchase.

Question: What are the main motivators for consumers in the 'rest of India' to shop online?
Cached response: The primary reasons consumers in the 'rest of India' opt for online shopping are the absence of physical stores for premium brands, stockouts of certain products in local stores, and a lack of knowledgeable staff in offline retail environments.

Question: Why do urban dwellers in India prefer to shop online?
Cached response: Urban dwellers are motivated to shop online due to the lack of discounts and special offers in physical stores and to avoid the large crowds often found in malls, particularly during weekends.

Question: How does social me